## 4. Voilà Dashboards

> Voilà turns Jupyter notebooks into standalone web applications.
> 
> Unlike the usual HTML-converted notebooks, each user connecting to the Voilà tornado application gets a dedicated Jupyter kernel which can execute the callbacks to changes in Jupyter interactive widgets.

- The [intro blog](https://blog.jupyter.org/and-voil%C3%A0-f6a2c08a4a93) provides an overview and resources for getting up and running with Voilà
- The [Voilà gallery](https://voila-gallery.org/) has a collection of examples built with Voilà and Jupyter widgets. 


#### running voila ([docs](https://voila.readthedocs.io/en/stable/using.html#using-voila)):
- you can run Voilà as a standalone server 
  
  ```
  voila notebook.ipynb
  ```
  
  
- or as a server extention by changing the url to `<url-of-my-server>/voila` (e.g. if you launched `jupyter lab` locally, and it was running at `http://localhost:8888/lab`, then then Voilà would be accessed at `http://localhost:8888/voila`.


- there is also a JupyterLab extention for previewing your dashboard
   
   ```
   jupyter labextension install @jupyter-voila/jupyterlab-preview
   ```

- by default, Voilà will strip out the source code from view. It can be displayed if the option `strip_sources` is set to False